In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [32]:
# Define the lyrics of the song
data = open('./data/irish-lyrics-eof.txt').read()

# Split the long string per line and put in a list
corpus = data.lower().split("\n")

# Preview the result
print(corpus[0])

come all ye maidens young and fair


In [33]:
# Initialize the Tokenizer class
tokenizer = Tokenizer()

# Generate the word index dictionary
tokenizer.fit_on_texts(corpus)

# Define the total words. You add 1 for the index `0` which is just the padding token.
total_words = len(tokenizer.word_index) + 1

print(f'total words: {total_words}')

total words: 2690


In [34]:
# Initialize the sequences list
input_sequences = []

# Loop over every line
for line in corpus:

    # Tokenize the current line
    token_list = tokenizer.texts_to_sequences([line])[0]

    # Loop over the line several times to generate the subphrases
    for i in range(1, len(token_list)):

        # Generate the subphrase
        n_gram_sequence = token_list[:i+1]

        # Append the subphrase to the sequences list
        input_sequences.append(n_gram_sequence)

# Get the length of the longest line
max_sequence_len = max([len(x) for x in input_sequences])

# Pad all sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create inputs and label by splitting the last token in the subphrases
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]

# Convert the label into one-hot arrays
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [35]:
# Get sample sentence
sentence = corpus[0].split()
print(f'sample sentence: {sentence}')

# Initialize token list
token_list = []

# Look up the indices of each word and append to the list
for word in sentence:
    token_list.append(tokenizer.word_index[word])

# Print the token list
print(token_list)

sample sentence: ['come', 'all', 'ye', 'maidens', 'young', 'and', 'fair']
[51, 12, 96, 1217, 48, 2, 69]


In [36]:
# Pick element
elem_number = 6

# Print token list and phrase
print(f'token list: {xs[elem_number]}')
print(f'decoded to text: {tokenizer.sequences_to_texts([xs[elem_number]])}')

token list: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 2]
decoded to text: ['and']


In [37]:
# Print label
print(f'one-hot label: {ys[elem_number]}')
print(f'index of label: {np.argmax(ys[elem_number])}')

one-hot label: [0. 0. 0. ... 0. 0. 0.]
index of label: 11


In [38]:
# Pick element
elem_number = 5

# Print token list and phrase
print(f'token list: {xs[elem_number]}')
print(f'decoded to text: {tokenizer.sequences_to_texts([xs[elem_number]])}')

# Print label
print(f'one-hot label: {ys[elem_number]}')
print(f'index of label: {np.argmax(ys[elem_number])}')

token list: [   0    0    0    0    0    0    0    0    0   51   12   96 1217   48
    2]
decoded to text: ['come all ye maidens young and']
one-hot label: [0. 0. 0. ... 0. 0. 0.]
index of label: 69


In [39]:
# Build the model
model = Sequential([
    Embedding(total_words, 100, input_length=max_sequence_len-1),
    Bidirectional(LSTM(150)),
    Dense(total_words, activation='softmax')
])

# Use categorical crossentropy because this is a multi-class problem
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 15, 100)           269000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 300)              301200    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 2690)              809690    
                                                                 
Total params: 1,379,890
Trainable params: 1,379,890
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Train the model
history = model.fit(xs, ys, epochs=100)

Epoch 1/100
377/377 [==============================] - 2s 5ms/step - loss: 4.6049 - accuracy: 0.1548
Epoch 2/100
377/377 [==============================] - 2s 6ms/step - loss: 4.2533 - accuracy: 0.1813
Epoch 3/100
377/377 [==============================] - 2s 5ms/step - loss: 3.9139 - accuracy: 0.2199
Epoch 4/100
377/377 [==============================] - 2s 5ms/step - loss: 3.5841 - accuracy: 0.2682
Epoch 5/100
377/377 [==============================] - 2s 5ms/step - loss: 3.2691 - accuracy: 0.3179
Epoch 6/100
377/377 [==============================] - 2s 5ms/step - loss: 2.9712 - accuracy: 0.3776
Epoch 7/100
377/377 [==============================] - 2s 5ms/step - loss: 2.6958 - accuracy: 0.4316
Epoch 8/100
377/377 [==============================] - 2s 5ms/step - loss: 2.4396 - accuracy: 0.4874
Epoch 9/100
377/377 [==============================] - 2s 5ms/step - loss: 2.2111 - accuracy: 0.5336
Epoch 10/100
377/377 [==============================] - 2s 5ms/step - loss: 2.0051 - accura

In [ ]:
import matplotlib.pyplot as plt

# Plot utility
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.show()

# Visualize the accuracy
plot_graphs(history, 'accuracy')

In [46]:
# Define seed text
seed_text = "Max is"

# Define total words to predict
next_words = 100

# Loop until desired length is reached
for _ in range(next_words):
    # Convert the seed text to a token sequence
    token_list = tokenizer.texts_to_sequences([seed_text])[0]

    # Pad the sequence
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

    # Feed to the model and get the probabilities for each index
    probabilities = model.predict(token_list)

    # Get the index with the highest probability
    predicted = np.argmax(probabilities, axis=-1)[0]

    # Ignore if index is 0 because that is just the padding.
    if predicted != 0:

        # Look up the word associated with the index.
        output_word = tokenizer.index_word[predicted]

        # Combine with the seed text
        seed_text += " " + output_word

# Print the result
print(seed_text)

Max is the sweetest of all dripping with honey to seen seen they might are thinking cry the rocky road to dublin dancing upon upon her shoulders fell in the time makes but ill be there on your eyes again well true did years well ask upon the spire of erins isle of fray blight loud reach and cold as seen as they they they take loud buy proud saxon cursed row de men who wee cold li li ivy hats sod ould bouys red dawning and your rest seen by the fray grow over the army they they they ra shore
